In [41]:
import pandas as pd
from collections import defaultdict

In [42]:
# define graph
def build_graph(link_list):
    graph = defaultdict(list)
    seen_edges = defaultdict(int)
    for src, dst, weight in link_list:
        seen_edges[(src, dst, weight)] += 1
        if seen_edges[(src, dst, weight)] > 1:  # checking for duplicated edge entries
            continue
        graph[src].append((dst, weight))
        graph[dst].append((src, weight))  # remove this line of edge list is directed
    return graph

In [43]:
# define Dijkstra algorithm
def dijkstra(graph, src, dst=None):
    nodes = []
    for n in graph:
        nodes.append(n)
        nodes += [x[0] for x in graph[n]]

    q = set(nodes)
    nodes = list(q)
    dist = dict()
    prev = dict()
    for n in nodes:
        dist[n] = float('inf')
        prev[n] = None

    dist[src] = 0

    while q:
        u = min(q, key=dist.get)
        q.remove(u)

        if dst is not None and u == dst:
            return dist[dst], prev

        for v, w in graph.get(u, ()):
            alt = dist[u] + w
            if alt < dist[v]:
                dist[v] = alt
                prev[v] = u

    return dist, prev

# generate path list based on parent points 'prev'
def find_path(pr, node):  
    p = []
    while node is not None:
        p.append(node)
        node = pr[node]
    return p[::-1]

In [44]:
if __name__ == "__main__":
    
    # read link file and create edges 
    network = pd.read_csv('input/network.csv')
    link_list = list()
    for index in network.index:
        temp = tuple()
        temp = temp + (network['NodeA'][index],)
        temp = temp + (network['NodeB'][index],)
        temp = temp + (network['Distance'][index],)
        link_list.append(temp)
    
    g = build_graph(link_list)

    print("=== Dijkstra ===")
    
    # run dijkstra algorithm to find pairwise shortest path
    # store the path found in a dictionary called 'pair'
    pair = dict()
    min_node = min(network['NodeA'].min(),network['NodeB'].min())
    max_node = max(network['NodeA'].max(),network['NodeB'].max())
    for i in range(min_node, max_node + 1):
        ds, prev = dijkstra(g, i)
        for k in ds:
            path = find_path(prev, k)
            print(i, " -> {}: shortest distance = {}, path = {}".format(k, ds[k], path))
            
            # for non-directional graph, only store the path for i->k, and omit k->i
            key = (i, k)
            key_p = (k, i)
            try:
                pair[key_p]
                exist = True
            except:
                exist = False

            if not exist:
                pair[key] = path

=== Dijkstra ===
1  -> 1: shortest distance = 0, path = [1]
1  -> 2: shortest distance = 8.4, path = [1, 2]
1  -> 3: shortest distance = 7.2, path = [1, 6, 3]
1  -> 4: shortest distance = 11.5, path = [1, 6, 3, 4]
1  -> 5: shortest distance = 12.0, path = [1, 6, 3, 5]
1  -> 6: shortest distance = 5.0, path = [1, 6]
2  -> 1: shortest distance = 8.4, path = [2, 1]
2  -> 2: shortest distance = 0, path = [2]
2  -> 3: shortest distance = 7.2, path = [2, 6, 3]
2  -> 4: shortest distance = 11.5, path = [2, 6, 3, 4]
2  -> 5: shortest distance = 12.0, path = [2, 6, 3, 5]
2  -> 6: shortest distance = 5.0, path = [2, 6]
3  -> 1: shortest distance = 7.2, path = [3, 6, 1]
3  -> 2: shortest distance = 7.2, path = [3, 6, 2]
3  -> 3: shortest distance = 0, path = [3]
3  -> 4: shortest distance = 4.3, path = [3, 4]
3  -> 5: shortest distance = 4.8, path = [3, 5]
3  -> 6: shortest distance = 2.2, path = [3, 6]
4  -> 1: shortest distance = 11.5, path = [4, 3, 6, 1]
4  -> 2: shortest distance = 11.5, path

In [45]:
# Read origin and destination file. Origin is index, destination is the column.
# For non-directional graph, the matrix should be symmetrical
OD = pd.read_csv('input/OD.csv', index_col = 'Zone')
OD

,1,2,3,4,5,6
Zone,,,,,,
1,0,1000,1100,400,1000,1300
2,1000,0,1050,700,1100,1200
3,1100,1050,0,1200,1150,1600
4,400,700,1200,0,800,400
5,1000,1100,1150,800,0,700
6,1300,1200,1600,400,700,0


In [47]:
# add a new column to links to store the traffic flow assigned to it
network['flow'] = network.index * 0
for i in range(min_node, max_node + 1):
    for j in range(min_node, max_node + 1):
        if i != j and i < j:
            
            # find shortest path and demand between node i and j 
            path = pair[(i, j)]
            demand = OD[str(i)][j]
            
            # assign traffic to flow to the links in the path
            for k in range(0, len(path) - 1):
                node1 = min(path[k], path[k + 1])
                node2 = max(path[k], path[k + 1])
                ind = (network['NodeA'] == node1) & (network['NodeB'] == node2)
                index = network.loc[ind].index[0]
                network.loc[index, 'flow'] += demand
                
network

,Links,NodeA,NodeB,Distance,flow
0,1,1,2,8.4,1000
1,2,1,5,12.6,0
2,3,1,6,5.0,3800
3,4,2,3,7.8,0
4,5,2,6,5.0,4050
5,6,3,4,4.3,2700
6,7,3,5,4.8,3950
7,8,3,6,2.2,8050
8,9,4,5,3.3,800
9,10,5,6,7.2,0
